<a href="https://colab.research.google.com/github/mahtabtech/VGG-and-MobileNet-with-FPN/blob/main/VGG_and_MobileNet_with_FPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Creating Feature Pyramid Networks (FPNs) using two other popular models: VGG-16 and MobileNetV2.**

# **FPN with VGG-16**

In [4]:
## Feature Pyramid Network (FPN) using VGG-16

import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

# Load the pre-trained VGG-16 model with the correct weights syntax
model_vgg = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)

# Extract feature maps from different layers of VGG-16
# VGG-16 has a simpler, sequential layer structure
# Shallow layer feature map (up to the 5th layer)
layer1_vgg = nn.Sequential(*list(model_vgg.features)[:5])  # Early stage feature map

# Middle layer feature map (up to the 10th layer)
layer2_vgg = nn.Sequential(*list(model_vgg.features)[:10])  # Middle stage feature map

# Deepest layer feature map (up to the 23rd layer, including all convolutional layers)
layer3_vgg = nn.Sequential(*list(model_vgg.features)[:23])  # Late stage feature map

# Example input tensor (e.g., a 224x224 image)
x_vgg = torch.randn(1, 3, 224, 224)

# Get feature maps from the different layers
feat1_vgg = layer1_vgg(x_vgg)  # Early stage feature map
feat2_vgg = layer2_vgg(x_vgg)  # Middle stage feature map
feat3_vgg = layer3_vgg(x_vgg)  # Late stage feature map

# Print the shapes of the feature maps
print(feat1_vgg.shape, feat2_vgg.shape, feat3_vgg.shape)

# Define the Feature Pyramid Network (FPN) for VGG-16
class FPN_VGG(nn.Module):
    def __init__(self):
        super(FPN_VGG, self).__init__()

        # 1x1 convolutions to reduce the number of channels to 256
        self.conv3 = nn.Conv2d(512, 256, kernel_size=1)  # For the deepest feature map
        self.conv2 = nn.Conv2d(128, 256, kernel_size=1)  # For the middle feature map
        self.conv1 = nn.Conv2d(64, 256, kernel_size=1)   # For the shallowest feature map

        # 3x3 convolution for further refinement
        self.conv_out = nn.Conv2d(256, 256, kernel_size=3, padding=1)

    def forward(self, feat1, feat2, feat3):
        # Convert feat3 to 256 channels (deepest feature map)
        p3 = self.conv3(feat3)

        # Upsample p3 and merge with the middle stage feature map (feat2)
        p3_upsampled = F.interpolate(p3, size=(feat2.shape[2], feat2.shape[3]), mode='nearest')
        p2 = self.conv2(feat2) + p3_upsampled
        p2 = self.conv_out(p2)

        # Upsample p2 and merge with the early stage feature map (feat1)
        p2_upsampled = F.interpolate(p2, size=(feat1.shape[2], feat1.shape[3]), mode='nearest')
        p1 = self.conv1(feat1) + p2_upsampled
        p1 = self.conv_out(p1)

        return p1, p2, p3

# Instantiate and run the FPN for VGG-16
fpn_vgg = FPN_VGG()
p1_vgg, p2_vgg, p3_vgg = fpn_vgg(feat1_vgg, feat2_vgg, feat3_vgg)

# Print the shapes of the output feature maps
print(p1_vgg.shape, p2_vgg.shape, p3_vgg.shape)

torch.Size([1, 64, 112, 112]) torch.Size([1, 128, 56, 56]) torch.Size([1, 512, 28, 28])
torch.Size([1, 256, 112, 112]) torch.Size([1, 256, 56, 56]) torch.Size([1, 256, 28, 28])


# **FPN with MobileNet**

In [5]:
## Feature Pyramid Network (FPN) using MobileNetV2

import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

# Load the pre-trained MobileNetV2 model with the correct weights syntax
model_mobilenet = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights.DEFAULT)

# Extract feature maps from different stages of MobileNetV2
# Shallow layer feature map (up to the 4th bottleneck)
layer1_mobilenet = nn.Sequential(*list(model_mobilenet.features)[:4])

# Middle layer feature map (up to the 7th bottleneck)
layer2_mobilenet = nn.Sequential(*list(model_mobilenet.features)[:7])

# Deepest layer feature map (up to the final bottleneck)
layer3_mobilenet = nn.Sequential(*list(model_mobilenet.features)[:14])

# Example input tensor (e.g., a 224x224 image)
x_mobilenet = torch.randn(1, 3, 224, 224)

# Get feature maps from the different layers
feat1_mobilenet = layer1_mobilenet(x_mobilenet)  # Early stage feature map
feat2_mobilenet = layer2_mobilenet(x_mobilenet)  # Middle stage feature map
feat3_mobilenet = layer3_mobilenet(x_mobilenet)  # Late stage feature map

# Print the shapes of the feature maps
print(feat1_mobilenet.shape, feat2_mobilenet.shape, feat3_mobilenet.shape)

# Define the Feature Pyramid Network (FPN) for MobileNetV2
class FPN_MobileNet(nn.Module):
    def __init__(self):
        super(FPN_MobileNet, self).__init__()

        # 1x1 convolutions to reduce the number of channels to 256
        self.conv3 = nn.Conv2d(96, 256, kernel_size=1)   # For the deepest feature map
        self.conv2 = nn.Conv2d(32, 256, kernel_size=1)   # For the middle feature map
        self.conv1 = nn.Conv2d(24, 256, kernel_size=1)   # For the shallowest feature map

        # 3x3 convolution for further refinement
        self.conv_out = nn.Conv2d(256, 256, kernel_size=3, padding=1)

    def forward(self, feat1, feat2, feat3):
        # Convert feat3 to 256 channels (deepest feature map)
        p3 = self.conv3(feat3)

        # Upsample p3 and merge with the middle stage feature map (feat2)
        p3_upsampled = F.interpolate(p3, size=(feat2.shape[2], feat2.shape[3]), mode='nearest')
        p2 = self.conv2(feat2) + p3_upsampled
        p2 = self.conv_out(p2)

        # Upsample p2 and merge with the early stage feature map (feat1)
        p2_upsampled = F.interpolate(p2, size=(feat1.shape[2], feat1.shape[3]), mode='nearest')
        p1 = self.conv1(feat1) + p2_upsampled
        p1 = self.conv_out(p1)

        return p1, p2, p3

# Instantiate and run the FPN for MobileNetV2
fpn_mobilenet = FPN_MobileNet()
p1_mobilenet, p2_mobilenet, p3_mobilenet = fpn_mobilenet(feat1_mobilenet, feat2_mobilenet, feat3_mobilenet)

# Print the shapes of the output feature maps
print(p1_mobilenet.shape, p2_mobilenet.shape, p3_mobilenet.shape)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 183MB/s]


torch.Size([1, 24, 56, 56]) torch.Size([1, 32, 28, 28]) torch.Size([1, 96, 14, 14])
torch.Size([1, 256, 56, 56]) torch.Size([1, 256, 28, 28]) torch.Size([1, 256, 14, 14])
